In [14]:
from IPython.core.display import HTML
import json

CRAWLER_DATA_FILE = '../law_dataset_crawler/out.json'
with open(CRAWLER_DATA_FILE, 'r') as crawler_data_file:
    data = json.load(crawler_data_file)
    

In [23]:
HTML(data[3]['text'])

Ключови фрази
Други искове за нарушения на авторски права * авторско право


In [181]:
import re

def clean_legal_text(text):
    text_copy = text
    
    try:
        num = re.search('(№)\s+(\d*)', text_copy)
        text_copy = re.sub('(№)\s+(\d*)', num.group(1)+num.group(2), text_copy)
    except:
        pass
    
    regex_rules = [
        (r'(?:(?!\n)\s){1,5}', ' '),
        (r'<!--[^>]*-->', ''),
        (r'(?:(?!\n)\s)(?:(?!\n)\s)+', '\n')
    ]
    
    for rule in regex_rules:
        text_copy = re.sub(rule[0], rule[1], text_copy)
    
    rules = [
        ('Р Е Ш Е Н И Е', 'РЕШЕНИЕ'),
        ('И М Е Т О', 'ИМЕТО'),
        ('Н А', 'НА'),
        ('Н А Р О Д А', 'НАРОДА'),
        ('В ИМЕТО НА НАРОДА', 'В ИМЕТО НА НАРОДА\n'),
        ('Р Е Ш И', 'РЕШИ'),
        ('О П Р Е Д Е Л И', 'ОПРЕДЕЛИ'),
        ('О П Р Е Д Е Л Е Н И Е', 'ОПРЕДЕЛЕНИЕ')
    ]
    
    for rule in rules:
        text_copy = text_copy.replace(rule[0], rule[1])
        
    return text_copy

In [197]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data[589]['text'])
keywords_text = soup.body.table.contents[1].getText()
keywords_list = [keyword.strip() for keyword in keywords_text.split("*")]

contents = soup.find_all("table")[1].getText()
cleaned_text = clean_legal_text(contents)

from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [199]:
cleaned_text

'\nРЕШЕНИЕ\n\n№288\n\nгр. София, 26.03.2018 г.\n\nВ ИМЕТО НА НАРОДА\n\n\nВЪРХОВНИЯТ КАСАЦИОНЕН СЪД НА РЕПУБЛИКА БЪЛГАРИЯ, НАКАЗАТЕЛНА КОЛЕГИЯ, второ наказателно отделение, в открито съдебно заседание на двадесет и втори ноември през две хиляди и седемнадесета година в състав:\n\n\nПРЕДСЕДАТЕЛ: ГАЛИНА ЗАХАРОВА\n\nЧЛЕНОВЕ: БИЛЯНА ЧОЧЕВА\n\nГАЛИНА ТОНЕВА\n\nпри секретар КР. ПАВЛОВА и с участието на прокурор АТ. ГЕБРЕВ разгледа докладваното от съдия ЗАХАРОВА наказателно дело №288/2017 г. по описа на ВКС, второ отделение, като за да се произнесе, взе предвид следното:\n\n Касационното производство е образувано на основание чл. 346, т. 1 от НПК по жалба на подсъдимата Н. А. С. чрез защитника й адв. Ю. Г. срещу присъда №288 от 22.06.2017 г. на Варненския апелативен съд (ВАС), наказателно отделение, постановено по ВНОХД №288/2017 г. по описа на същия съд.\nВ жалбата на подсъдимата са релевирани всички касационни основания по чл. 348, ал. 1, т. 1 – т. 3 от НПК.\nВ писмено допълнение към касацио

In [276]:
nltk_sentences = sent_tokenize(cleaned_text)
nltk_sentences

['\nРЕШЕНИЕ\n\n№288\n\nгр.',
 'София, 26.03.2018 г.\n\nВ ИМЕТО НА НАРОДА\n\n\nВЪРХОВНИЯТ КАСАЦИОНЕН СЪД НА РЕПУБЛИКА БЪЛГАРИЯ, НАКАЗАТЕЛНА КОЛЕГИЯ, второ наказателно отделение, в открито съдебно заседание на двадесет и втори ноември през две хиляди и седемнадесета година в състав:\n\n\nПРЕДСЕДАТЕЛ: ГАЛИНА ЗАХАРОВА\n\nЧЛЕНОВЕ: БИЛЯНА ЧОЧЕВА\n\nГАЛИНА ТОНЕВА\n\nпри секретар КР.',
 'ПАВЛОВА и с участието на прокурор АТ.',
 'ГЕБРЕВ разгледа докладваното от съдия ЗАХАРОВА наказателно дело №288/2017 г. по описа на ВКС, второ отделение, като за да се произнесе, взе предвид следното:\n\n Касационното производство е образувано на основание чл.',
 '346, т.',
 '1 от НПК по жалба на подсъдимата Н. А. С. чрез защитника й адв.',
 'Ю. Г. срещу присъда №288 от 22.06.2017 г. на Варненския апелативен съд (ВАС), наказателно отделение, постановено по ВНОХД №288/2017 г. по описа на същия съд.',
 'В жалбата на подсъдимата са релевирани всички касационни основания по чл.',
 '348, ал.',
 '1, т.',
 '1 – т.',
 

In [277]:
len(nltk_sentences)

452

In [247]:
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [250]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True

In [268]:
punkt_train_data = ""
for i in range(400):
    soup = BeautifulSoup(data[i]['text'])
    keywords_text = soup.body.table.contents[1].getText()
    keywords_list = [keyword.strip() for keyword in keywords_text.split("*")]

    contents = soup.find_all("table")[1].getText()
    punkt_train_data += clean_legal_text(contents)

In [269]:
len(punkt_train_data)

4718056

In [270]:
trainer.train(punkt_train_data)

In [271]:
trainer.get_params().abbrev_types

{'.о',
 '/2008г',
 '/2009г',
 '/2010г',
 '/x',
 '/вж',
 '/к',
 '/л',
 '/л.л',
 '/лв',
 '/обн',
 '/ред',
 '/срвн',
 '/т',
 '/ф',
 '/чл',
 '000лв',
 '01.01.2008г',
 '01.07.1994г',
 '01.07.2007г',
 '01.07.2011г',
 '01.07.2012г',
 '01.10.2002г',
 '03.05.2012г',
 '03.06.2011г',
 '03.07.2014г',
 '03.08.2007год',
 '03.09.2010г',
 '03.11.2011г',
 '04.01.2011г',
 '04.06.1998год',
 '04.06.2013г',
 '04.07.2008-08.03.2010г',
 '04.10.2012г',
 '04.10ч',
 '04.12.2013г',
 '04.36ч',
 '05.03.2012г',
 '05.05.2011г',
 '05.07.2011г',
 '05.хі.2013г',
 '06.01.2014г',
 '06.03.2012г',
 '06.04.2000г',
 '06.07.15г',
 '06.08.2010г',
 '06.11.2018г',
 '07.02.2005г',
 '07.10.2013г',
 '08.02.2008г',
 '08.11.1998год',
 '08.12.07г',
 '08.12.2005г',
 '08.12.2015г',
 '09.07.2015г',
 '09.08.2005г',
 '09.09.2010г',
 '09.10.2014г',
 '1,т',
 '1.04.2008г',
 '1.06.2010г',
 '1.10.2004г',
 '10.01.2017г',
 '10.02.2006г',
 '10.02.2011г',
 '10.03.2000г',
 '10.04.2014г',
 '10.06.2004г',
 '10.06.2014г',
 '10.07.2008г',
 '10.11.2013г'

In [263]:
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [273]:
improved_sentences = tokenizer.tokenize(cleaned_text)

In [274]:
len(improved_sentences)

153

In [286]:
improved_sentences[3:6]

['В писмено допълнение към касационната жалба (с вх. №288 от 21.08.2017 г. на ВАС) са изложени съображения за незаконосъобразно осъждане на касатора С. по чл. 212, ал. 5 от НК.',
 'Твърди се, че не бил налице критерият особено тежък случай на деянието, поради което то следвало да бъде преквалифицирано по чл. 212, ал. 4 от НК, както първоначално процедирал първоинстанционният съд.',
 'Мотивите на въззивния съдебен акт в тази насока били противоречиви, като съдът сам посочил, че част от разсъжденията му относно документооборота не касаели правната квалификация.']

In [308]:
import jsonlines

with jsonlines.open('output.jsonl', mode='w') as writer:
    for i in range(50):
        soup = BeautifulSoup(data[i]['text'])
        keywords_text = soup.body.table.contents[1].getText()
        keywords_list = [keyword.strip() for keyword in keywords_text.split("*")]

        contents = soup.find_all("table")[1].getText()
        cleaned_legal_text = clean_legal_text(contents)
        
        splitted_sentences = tokenizer.tokenize(cleaned_legal_text)
        
        for j in range(0, len(splitted_sentences), 3):
            sep = ' '
            doc_text = sep.join(splitted_sentences[j:j+3])
            doc = {'text':doc_text}

            writer.write(doc)

In [309]:
!tail output.jsonl

{"text": "Поддържа също така,че съображенията за връщане на въззивната жалба са нехуманни с оглед здравословното й състояние. Поддържат,че жалбата следва да бъде допусната до касационно обжалване на основание чл.280, ал.1,т.3 ГПК,тъй като с обжалваното определение съдът се е произнесъл по въпроса задължен ли е съдът да уведоми страната,че определеният от закона срок за отстраняване на констатираните нередовности на подадена въззивна жалба е продължен. Ответникът по касационна жалба П. С. Л. не изразява становище."}
{"text": "Частната жалба е подадена в срока по чл.275,ал.1 ГПК и е допустима по смисъла на чл.280,ал.3 ГПК. Налице са и предпоставките за допускане на касационното обжалване с оглед необходимостта да бъде дадено тълкуване на разпоредбата на чл.63 ГПК. В обжалваното определение въззивният съд се е произнесъл по процесуален въпрос, касаещ необходимостта от съобщаване на акта на съда,с който по искане на страната е продължен определен в закона срок,за последиците от съобщаванет